Aim 1: Plot accelerometer data one participant across multiple time scales -- a single hour, a single day, the study duration (1 week)

In [1]:
%load_ext autoreload
%autoreload 2 


import pyarrow.parquet as pq
import pyarrow as pa
from datetime import timedelta
from collections import defaultdict
from pathlib import Path
import re
import random
import math
import warnings
import logging
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd
import pytz
import matplotlib.pyplot as plt

In [ ]:
# Load all 24 hourly dataframes for participant 3si9xdvl on 2022-03-26
base_path = "/n/home01/egraff/sample_imputation/data/raw/3si9xdvl/accelerometer/"
date = "2022-03-26"

# Dictionary to store all dataframes
hourly_dfs = {}

# Load all 24 hours (00 to 23)
for hour in range(24):
    hour_str = f"{hour:02d}"  # Format as 2-digit string with leading zero
    filename = f"{date} {hour_str}_00_00+00_00.csv"
    filepath = base_path + filename
    
    try:
        df = pd.read_csv(filepath)
        hourly_dfs[f"hr_{hour+1}_df"] = df  # hr_1_df through hr_24_df
        print(f"hr_{hour+1}_df ({hour_str}:00): {df.shape[0]} rows")
    except FileNotFoundError:
        print(f"File not found: {filename}")
    except Exception as e:
        print(f"Error loading {filename}: {e}")

# Also create individual variables for easier access
hr_1_df = hourly_dfs.get("hr_1_df")
hr_2_df = hourly_dfs.get("hr_2_df")
hr_3_df = hourly_dfs.get("hr_3_df")
hr_4_df = hourly_dfs.get("hr_4_df")
hr_5_df = hourly_dfs.get("hr_5_df")
hr_6_df = hourly_dfs.get("hr_6_df")
hr_7_df = hourly_dfs.get("hr_7_df")
hr_8_df = hourly_dfs.get("hr_8_df")
hr_9_df = hourly_dfs.get("hr_9_df")
hr_10_df = hourly_dfs.get("hr_10_df")
hr_11_df = hourly_dfs.get("hr_11_df")
hr_12_df = hourly_dfs.get("hr_12_df")
hr_13_df = hourly_dfs.get("hr_13_df")
hr_14_df = hourly_dfs.get("hr_14_df")
hr_15_df = hourly_dfs.get("hr_15_df")
hr_16_df = hourly_dfs.get("hr_16_df")
hr_17_df = hourly_dfs.get("hr_17_df")
hr_18_df = hourly_dfs.get("hr_18_df")
hr_19_df = hourly_dfs.get("hr_19_df")
hr_20_df = hourly_dfs.get("hr_20_df")
hr_21_df = hourly_dfs.get("hr_21_df")
hr_22_df = hourly_dfs.get("hr_22_df")
hr_23_df = hourly_dfs.get("hr_23_df")
hr_24_df = hourly_dfs.get("hr_24_df")

In [8]:
hr8_df.tail()

,timestamp,UTC time,accuracy,x,y,z
18093,1648285178322,2022-03-26T08:59:38.322,unknown,0.047119,-0.031876,-1.005508
18094,1648285178421,2022-03-26T08:59:38.421,unknown,0.047882,-0.031998,-1.004623
18095,1648285178521,2022-03-26T08:59:38.521,unknown,0.047852,-0.030685,-1.005112
18096,1648285178620,2022-03-26T08:59:38.620,unknown,0.047806,-0.031479,-1.004639
18097,1648285178720,2022-03-26T08:59:38.720,unknown,0.047974,-0.031754,-1.005722


In [9]:
hr9_df.head()

,timestamp,UTC time,accuracy,x,y,z
0,1648285238792,2022-03-26T09:00:38.792,unknown,0.048126,-0.031631,-1.006866
1,1648285238891,2022-03-26T09:00:38.891,unknown,0.047867,-0.031448,-1.005661
2,1648285238991,2022-03-26T09:00:38.991,unknown,0.047729,-0.030899,-1.004883
3,1648285239090,2022-03-26T09:00:39.090,unknown,0.047562,-0.030396,-1.005432
4,1648285239190,2022-03-26T09:00:39.190,unknown,0.047379,-0.030380,-1.004913


In [13]:
print(f"{hr5_df.shape[0]}, {hr6_df.shape[0]}, {hr7_df.shape[0]}, {hr8_df.shape[0]}, {hr9_df.shape[0]}")

11438, 18081, 18100, 18098, 18090


In [13]:
col_to_move = 'UTC time'
cols = [col_to_move] + [c for c in hr_df.columns if c != col_to_move]
hr_df = hr_df[cols]
hr_df.head()
# df_acc = hr_df[["x", ""]]


,UTC time,timestamp,accuracy,x,y,z
0,2022-03-26T09:00:38.792,1648285238792,unknown,0.048126,-0.031631,-1.006866
1,2022-03-26T09:00:38.891,1648285238891,unknown,0.047867,-0.031448,-1.005661
2,2022-03-26T09:00:38.991,1648285238991,unknown,0.047729,-0.030899,-1.004883
3,2022-03-26T09:00:39.090,1648285239090,unknown,0.047562,-0.030396,-1.005432
4,2022-03-26T09:00:39.190,1648285239190,unknown,0.047379,-0.030380,-1.004913


In [14]:
hr_df[["x", "y", "z"]].describe()

,x,y,z
count,18090.000000,18090.000000,18090.000000
mean,0.045451,-0.029257,-1.005472
std,0.001526,0.001241,0.000643
min,0.030258,-0.045441,-1.011627
25%,0.044189,-0.029907,-1.005859
50%,0.045441,-0.029068,-1.005463
75%,0.046219,-0.028458,-1.005066
max,0.055161,-0.010208,-0.992264
